<a href="https://colab.research.google.com/github/yadavrishikesh/BayesNF/blob/main/Portland_HourlyTraffic_DataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary packages
!pip install -q bayesnf cartopy contextily geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.0 MB/s eta 0:00:00


In [2]:
import warnings
warnings.simplefilter('ignore')

import contextily as ctx
import geopandas as gpd
import jax
import numpy as np
import pandas as pd
import os
import datetime
from urllib.parse import urlparse
from bayesnf.spatiotemporal import BayesianNeuralFieldMAP

In [3]:
def run_portland_model(train_url, test_url):
    """
    Trains a Bayesian Neural Field MAP model on specified Portland train data and tests on test data.

    Parameters:
    - train_url (str): URL to the training dataset CSV file.
    - test_url (str): URL to the test dataset CSV file.

    Returns:
    - predictions_df (DataFrame): DataFrame containing true values, predicted mean, and quantiles.
    """
    # Determine base directory from train_url and create unique outputs directory
    train_name = os.path.basename(urlparse(train_url).path).split('.')[0]
    test_name = os.path.basename(urlparse(test_url).path).split('.')[0]
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = os.path.join(os.getcwd(), 'outputs', f'{train_name}_{test_name}_{timestamp}')
    os.makedirs(output_dir, exist_ok=True)

    # Download and load datasets
    !wget -q {train_url} -O train_data.csv
    !wget -q {test_url} -O test_data.csv
    df_train = pd.read_csv('train_data.csv', index_col=0, parse_dates=['datetime'])
    df_test = pd.read_csv('test_data.csv', index_col=0, parse_dates=['datetime'])

    # Log-transform response in training data
    df_train_log = df_train.copy()
    df_train_log['response'] = np.log1p(df_train_log['response'])

    # Define and train the Bayesian Neural Field model
    model = BayesianNeuralFieldMAP(
        width=512,
        depth=2,
        freq='H',
        seasonality_periods=['D'],
        num_seasonal_harmonics=[2],
        feature_cols=['datetime', 'lat', 'lon', 'speed', 'occupancy'],
        target_col='response',
        observation_model='NORMAL',
        timetype='index',
        standardize=['lat', 'lon', 'speed', 'occupancy'],
    )

    model = model.fit(df_train_log,
                      seed=jax.random.PRNGKey(0),
                      ensemble_size=32,
                      num_epochs=2500)

    # Process test data
    df_test_cleaned = df_test.dropna(subset=['response'])
    df_test_cleaned_log = df_test_cleaned.copy()
    df_test_cleaned_log['response'] = np.log1p(df_test_cleaned_log['response'])

    # Make predictions
    yhat, yhat_quantiles = model.predict(df_test_cleaned, quantiles=(0.025, 0.5, 0.975))

    # Compile results into DataFrame
    predictions_df = pd.DataFrame({
        'datetime': df_test_cleaned.index,
        'True_Values': df_test_cleaned['response'],
        'Predicted_Mean': yhat_quantiles[1],
        'Quantile_0.025': yhat_quantiles[0],
        'Quantile_0.975': yhat_quantiles[2]
    })

    # Save output in the unique outputs directory
    output_path = os.path.join(output_dir, 'predictions_output.csv')
    predictions_df.to_csv(output_path, index=False)

    return predictions_df

In [4]:
# Spatial predictions
train_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/train_data_Portland_hourly_pred-type_spatIntpl.csv'
test_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/test_data_Portland_hourly_pred-type_spatIntpl.csv'
predictions_df = run_portland_model(train_data_url, test_data_url)
predictions_df.head()


,datetime,True_Values,Predicted_Mean,Quantile_0.025,Quantile_0.975
1,1,1097,7.479935,7.169591,7.769511
2,2,1059,7.458578,7.135786,7.759977
3,3,779,7.238156,6.955547,7.510957
4,4,690,7.118578,6.839669,7.393750
5,5,668,7.066780,6.801536,7.333193


In [5]:
# Forecasting
train_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/train_data_Portland_hourly_pred-type_forecast.csv'
test_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/test_data_Portland_hourly_pred-type_forecast.csv'
predictions_df = run_portland_model(train_data_url, test_data_url)
predictions_df.head()

,datetime,True_Values,Predicted_Mean,Quantile_0.025,Quantile_0.975
1,1,306,5.594604,5.351007,5.846370
2,2,180,5.028346,4.764715,5.306126
3,3,195,4.781106,4.505291,5.079773
4,4,274,5.038489,4.753059,5.315789
5,5,575,5.920913,5.630860,6.187080


In [6]:
# Space-time predictions
train_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/train_data_Portland_hourly_pred-type_spaceTime.csv'
test_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/test_data_Portland_hourly_pred-type_spaceTime.csv'
predictions_df = run_portland_model(train_data_url, test_data_url)
predictions_df.head()

,datetime,True_Values,Predicted_Mean,Quantile_0.025,Quantile_0.975
1,1,364,6.591866,6.278629,6.856924
2,2,293,6.124856,5.823220,6.395465
3,3,324,6.141189,5.832623,6.419325
4,4,561,6.581886,6.288296,6.849253
5,5,1137,7.296647,6.970715,7.600110


In [8]:
# Mount Google Drive to access it from Colab
from google.colab import drive
drive.mount('/content/drive')

# Copy the 'outputs' directory to Google Drive
!cp -r /content/outputs /content/drive/MyDrive/BayesNF/outputs_Portland_hourly


Mounted at /content/drive
